Khi ai đó đề cập đến "Question Answering" như một ứng dụng của BERT, những gì họ thực sự đang đề cập đến là áp dụng BERT cho bộ dữ liệu Stanfod Question Answering. (SquAD).

Nhiệm vụ được đặt ra bởi điểm chuẩn SQuAD hơi khác một chút so với bạn nghĩ.  Đưa ra một câu hỏi và một đoạn văn bản chứa câu trả lời, BERT cần đánh dấu "khoảng" văn bản tương ứng với câu trả lời đúng.

In [1]:
import torch

In [2]:
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [3]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [4]:
question = "How many parameters does BERT-large have?"
answer_text = "BERT-large is really big... it has 24-layers and an embedding size of 1,024, for a total of 340M parameters! Altogether it is 1.34GB, so expect it to take a couple minutes to download to your Colab instance."

In [5]:
# Appply the tokenizer to the input text, treating them as a text-pair

input_ids = tokenizer.encode(question, answer_text)

print(input_ids)

[101, 2129, 2116, 11709, 2515, 14324, 1011, 2312, 2031, 1029, 102, 14324, 1011, 2312, 2003, 2428, 2502, 1012, 1012, 1012, 2009, 2038, 2484, 1011, 9014, 1998, 2019, 7861, 8270, 4667, 2946, 1997, 1015, 1010, 6185, 2549, 1010, 2005, 1037, 2561, 1997, 16029, 2213, 11709, 999, 10462, 2009, 2003, 1015, 1012, 4090, 18259, 1010, 2061, 5987, 2009, 2000, 2202, 1037, 3232, 2781, 2000, 8816, 2000, 2115, 15270, 2497, 6013, 1012, 102]


In [6]:

tokens = tokenizer.convert_ids_to_tokens(input_ids)

for token, id in zip(tokens, input_ids):
    print('{:<12} {:>6}'.format(token, id))

[CLS]           101
how            2129
many           2116
parameters    11709
does           2515
bert          14324
-              1011
large          2312
have           2031
?              1029
[SEP]           102
bert          14324
-              1011
large          2312
is             2003
really         2428
big            2502
.              1012
.              1012
.              1012
it             2009
has            2038
24             2484
-              1011
layers         9014
and            1998
an             2019
em             7861
##bed          8270
##ding         4667
size           2946
of             1997
1              1015
,              1010
02             6185
##4            2549
,              1010
for            2005
a              1037
total          2561
of             1997
340           16029
##m            2213
parameters    11709
!               999
altogether    10462
it             2009
is             2003
1              1015
.              1012


In [7]:
# search the input_ids for the first instance of the [SEP] token
sep_index = input_ids.index(tokenizer.sep_token_id)

# The number of segment A tokens inlcude the [SEP] token itself
num_seg_a = sep_index + 1

# The remainer are segment B
num_seg_b = len(input_ids) - num_seg_a

# construct the list of 0s and 1s
segment_ids = [0]*num_seg_a + [1]*num_seg_b

# There should be a segment_id for every input token

assert len(segment_ids) == len(input_ids)

In [8]:
# Run our example through the model.
start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                 token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer_text


In [9]:
start_scores

tensor([[-6.4849, -6.4358, -8.1077, -8.8489, -7.8751, -8.0522, -8.4684, -8.5295,
         -7.7074, -9.2464, -6.4849, -2.7303, -6.3473, -5.7299, -7.7780, -7.0391,
         -6.3331, -7.3153, -7.3048, -7.4121, -2.2534, -5.3971, -0.9424, -7.3584,
         -5.4575, -7.0769, -4.4887, -3.9272, -5.6967, -5.9506, -5.0059, -5.9812,
          0.0530, -5.5968, -4.7093, -4.5750, -6.1786, -2.2294, -0.1904, -0.2327,
         -2.7331,  6.4256, -2.6543, -4.5655, -4.9872, -4.9834, -5.9110, -7.8402,
         -1.8986, -7.2123, -4.1543, -6.2354, -8.0953, -7.2329, -6.4411, -6.8384,
         -8.1032, -7.0570, -7.7332, -6.8711, -7.1045, -8.2966, -6.1939, -8.0817,
         -7.5501, -5.9695, -8.1007, -6.8849, -8.2273, -6.4850]],
       grad_fn=<SqueezeBackward1>)

In [10]:
# Find the tokens with the highest `start` and `end` scores.
answer_start = torch.argmax(start_scores)
answer_end = torch.argmax(end_scores)

# Combine the tokens in the answer and print it out.
answer = ' '.join(tokens[answer_start:answer_end+1])

print('Answer: "' + answer + '"')

Answer: "340 ##m"


Lưu ý bên: Hơi ngây thơ khi chọn điểm cao nhất cho phần bắt đầu và kết thúc - điều gì sẽ xảy ra nếu nó dự đoán một từ kết thúc trước từ bắt đầu ?!  Cách triển khai đúng là chọn tổng điểm cao nhất cho kết thúc> = bắt đầu.

In [11]:
# Start with the first token.
answer = tokens[answer_start]

# Select the remaining answer tokens and join them with whitespace.
for i in range(answer_start + 1, answer_end + 1):
    
    # If it's a subword token, then recombine it with the previous token.
    if tokens[i][0:2] == '##':
        answer += tokens[i][2:]
    
    # Otherwise, add a space then the token.
    else:
        answer += ' ' + tokens[i]

print('Answer: "' + answer + '"')

Answer: "340m"
